In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

def transcribe_audio(audio_path: str) -> str:
    audio_file = open("/path/to/file/speech.mp3", "rb")
    transcription = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file,
    )
    print(transcription.text)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/file/speech.mp3'

In [27]:
from pydub import AudioSegment 
  
song = AudioSegment.from_mp3("D:\\telelyzerv2\\experiment\\exp_data\\3ebf0da3-8930-4d66-91e4-5971d6fc4258.mp3")

first_5_seconds = song[:30000]

first_5_seconds.export("first_5_seconds.mp3", format="mp3") 


<_io.BufferedRandom name='first_5_seconds.mp3'>

In [8]:
!pip install librosa

In [12]:
import librosa

audio_data, sample_rate = librosa.load("D:\\telelyzerv2\\experiment\\exp_data\\53501fef-ef8b-4a08-b171-bcd7846ea639.mp3")

duration = librosa.get_duration(y=audio_data, sr=sample_rate)
print(duration / 2)

364.4640136054422


In [16]:
import math

duration_round_off = math.ceil(duration / 2)

duration_start = 0
duration_end = 2000

song = AudioSegment.from_mp3("D:\\telelyzerv2\\experiment\\exp_data\\3ebf0da3-8930-4d66-91e4-5971d6fc4258.mp3")

audio_segments = []
for i in range(0, duration_round_off):
    fragment = song[duration_start:duration_end]
    audio_segments.append(fragment)

    duration_start += 2000
    duration_end += 2000

In [21]:
sample_rate

22050

In [ ]:
# Running whisper on these audio segments. 

In [19]:
len(audio_segments), audio_segments

(365,
  <pydub.audio_segment.AudioSegment at 0x2a6549afc20>])

In [26]:
import torch
from transformers import pipeline

# path to the audio file to be transcribed
audio = "D:\\telelyzerv2\\experiment\\exp_data\\3ebf0da3-8930-4d66-91e4-5971d6fc4258.mp3"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transcribe = pipeline(task="automatic-speech-recognition", model="vasista22/whisper-hindi-large-v2", chunk_length_s=30, device=device)
transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="hi", task="transcribe")

print('Transcription: ', transcribe(audio)["text"])


c:\Python\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\athar\.cache\huggingface\hub\models--vasista22--whisper-hindi-large-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Python\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


IndexError: index -2 is out of bounds for dimension 0 with size 0

In [25]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from pydub import AudioSegment
import numpy as np
import torch

# Load processor and model
processor = AutoProcessor.from_pretrained("shripadbhat/whisper-medium-hi")
model = AutoModelForSpeechSeq2Seq.from_pretrained("shripadbhat/whisper-medium-hi")

# Assuming audio_segments is a list of pydub AudioSegment objects
for audio_segment in audio_segments:
    # Ensure the segment is not empty
    if len(audio_segment) == 0:
        print("Warning: Skipping empty audio segment.")
        continue
    
    # Resample to 16kHz
    audio_segment_16k = audio_segment.set_frame_rate(16000)
    samples = np.array(audio_segment_16k.get_array_of_samples()).astype(np.float32)
    
    # Process the audio and convert it to the right format for the model
    inputs = processor(samples, sampling_rate=16000, return_tensors="pt", padding=True, truncation=True)

    # Run the model for transcription
    with torch.no_grad():
        generated_ids = model.generate(inputs["input_features"], suppress_tokens=[])


    # Decode the output
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(transcription)


IndexError: index -2 is out of bounds for dimension 0 with size 0